In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install laserembeddings

     |████████████████████████████████| 51kB 2.7MB/s 
     |████████████████████████████████| 860kB 7.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=4fcb4ba0f1edcc7aacab97b9dcee9348af2e2f2916cb698e5a581b518e16fc80
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [ ]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [ ]:
import pandas as pd
import numpy as np
import os
from laserembeddings import Laser
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import pickle

In [ ]:
lang = ['Arabic','English','French']
cols = []
for i in range(1024):
  cols.append(i)

In [ ]:
def preprocessing(language):
    path = 'drive/My Drive/CS695'
    data = pd.read_csv(os.path.join(path,language+'.csv'))
    data_to_list = data['tweet'].to_list()

    laser = Laser()
    embeddings = laser.embed_sentences(data_to_list, lang='en') 

    df1=pd.DataFrame(embeddings)
    df2 = pd.DataFrame(data['label'])

    sss1 = StratifiedShuffleSplit(n_splits=1,test_size=0.3,random_state=0)
    a = list(sss1.split(df1, df2))
    train_x,interX = df1.iloc[a[0][0],:],df1.iloc[a[0][1],:]
    train_y,interY = df2.iloc[a[0][0],:],df2.iloc[a[0][1],:]

    sss2 = StratifiedShuffleSplit(n_splits=1,test_size=0.35,random_state=0)
    a = list(sss2.split(interX, interY))
    test_x,val_x = interX.iloc[a[0][0],:],interX.iloc[a[0][1],:]
    test_y,val_y = interY.iloc[a[0][0],:],interY.iloc[a[0][1],:]

    return train_x, test_x, val_x, train_y, test_y, val_y 

In [ ]:
def model_for_zeroshot(language, c, seed):
    
    df_ = pd.DataFrame()
    for l in lang:
        
        if l!= language:
            train_x, t_, v_, train_y, ty_, vy_ = preprocessing(l)
            
            _ = pd.concat([train_x, train_y], axis=1)
            
            df_ = pd.concat([_.reset_index(drop=True), df_], axis=0)
            
            
        elif l==language:
            t_, test_x, val_x, t_, test_y, val_y = preprocessing(language)
            
            
    df_ = df_.sample(frac=1,random_state=seed).reset_index(drop=True)

    df_x = df_[cols]
    df_y = df_['label']
  
    model = LogisticRegression(C=c,solver='lbfgs',class_weight='balanced',random_state=seed)
    model.fit(df_x, df_y)
   

    test_predict = model.predict(test_x)
    val_predict = model.predict(val_x)

    test_f1_score = f1_score(test_y, test_predict, average='macro')
    val_f1_score = f1_score(val_y, val_predict, average='macro')
    test_acc = accuracy_score(test_y, test_predict)
    val_acc = accuracy_score(val_y, val_predict)

  #saved_model = pickle.dumps(model) 

    print('Language:',language, '    ','Model loaded: Zero Shot', '    ','c:',c,'    ','seed:',seed)
    print('\n F1 macro score for test:', test_f1_score)
    print('\n F1 macro score for val:', val_f1_score)
    print('\n Accuracy for test:', test_acc)
    print('\n Accuracy for val:', val_acc)
    
    return test_f1_score

In [ ]:
for language in ['Arabic','English','French']:
  f1_list = []
  for c in [0.01,0.1,1,10]:
    for seeds in [2018,2019,2020,2021,2022]:
      seed = seeds
      np.random.seed(seed)
      a = model_for_zeroshot(language,c,seed)
      f1_list.append(a)
  
  print(f1_list)
  avg = sum(f1_list)/len(f1_list)

  print('Average F1 macro:',avg,'    ', 'language:',language)

Language: Arabic      Model loaded: Zero Shot      c: 0.01      seed: 2018

 F1 macro score for test: 0.6311284119338336

 F1 macro score for val: 0.6309072913229957

 Accuracy for test: 0.6899911426040743

 Accuracy for val: 0.6973684210526315
Language: Arabic      Model loaded: Zero Shot      c: 0.01      seed: 2019

 F1 macro score for test: 0.6311284119338336

 F1 macro score for val: 0.6309072913229957

 Accuracy for test: 0.6899911426040743

 Accuracy for val: 0.6973684210526315
Language: Arabic      Model loaded: Zero Shot      c: 0.01      seed: 2020

 F1 macro score for test: 0.6311284119338336

 F1 macro score for val: 0.6309072913229957

 Accuracy for test: 0.6899911426040743

 Accuracy for val: 0.6973684210526315
Language: Arabic      Model loaded: Zero Shot      c: 0.01      seed: 2021

 F1 macro score for test: 0.6311284119338336

 F1 macro score for val: 0.6309072913229957

 Accuracy for test: 0.6899911426040743

 Accuracy for val: 0.6973684210526315
Language: Arabic    

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Language: Arabic      Model loaded: Zero Shot      c: 1      seed: 2018

 F1 macro score for test: 0.6104715137236215

 F1 macro score for val: 0.6252801434334379

 Accuracy for test: 0.6536758193091231

 Accuracy for val: 0.6743421052631579


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Language: Arabic      Model loaded: Zero Shot      c: 1      seed: 2019

 F1 macro score for test: 0.6104715137236215

 F1 macro score for val: 0.6276814328744214

 Accuracy for test: 0.6536758193091231

 Accuracy for val: 0.6759868421052632


...

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Language: French      Model loaded: Zero Shot      c: 10      seed: 2020

 F1 macro score for test: 0.4062091503267974

 F1 macro score for val: 0.4448081469358065

 Accuracy for test: 0.4177215189873418

 Accuracy for val: 0.4496124031007752


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Language: French      Model loaded: Zero Shot      c: 10      seed: 2021

 F1 macro score for test: 0.4062091503267974

 F1 macro score for val: 0.4448081469358065

 Accuracy for test: 0.4177215189873418

 Accuracy for val: 0.4496124031007752
Language: French      Model loaded: Zero Shot      c: 10      seed: 2022

 F1 macro score for test: 0.4148148148148148

 F1 macro score for val: 0.4448081469358065

 Accuracy for test: 0.42616033755274263

 Accuracy for val: 0.4496124031007752
[0.44135714285714284, 0.44135714285714284, 0.44135714285714284, 0.44135714285714284, 0.44135714285714284, 0.436770848336454, 0.436770848336454, 0.436770848336454, 0.436770848336454, 0.436770848336454, 0.3990001449065353, 0.3990001449065353, 0.3990001449065353, 0.3990001449065353, 0.3990001449065353, 0.4161715693377771, 0.41120681838788653, 0.4062091503267974, 0.4062091503267974, 0.4148148148148148]
Average F1 macro: 0.42201260918473676      language: French


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
